In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !cp '/content/drive/MyDrive/f_data.zip' /content/

In [ ]:
# !cp '/content/drive/MyDrive/f_data.zip' /content/ # Copy the file
# !echo "Original file size in Drive:"
# !stat -c%s '/content/drive/MyDrive/f_data.zip'  # Get original size
# !echo "Copied file size in Colab:"
# !stat -c%s '/content/f_data.zip'  # Get copied size

Original file size in Drive:
3431067981
Copied file size in Colab:
3431067981


In [ ]:
# !zip -FF /content/f_data.zip --out /content/repaired_f_data.zip

Fix archive (-FF) - salvage what can
 Found end record (EOCDR) - says expect single disk archive
Scanning for entries...
 copying: f_data/  (0 bytes)
 copying: f_data/Healthy/  (0 bytes)
 copying: f_data/Healthy/0 (2).jpg  (23563 bytes)
 copying: f_data/Healthy/0.jpg  (23563 bytes)
 copying: f_data/Healthy/1 (2).jpg  (23610 bytes)
 copying: f_data/Healthy/1.jpg  (23610 bytes)
 copying: f_data/Healthy/10 (2).jpg  (24173 bytes)
 copying: f_data/Healthy/10.jpg  (24173 bytes)
 copying: f_data/Healthy/11 (2).jpg  (17937 bytes)
 copying: f_data/Healthy/11.jpg  (17937 bytes)
 copying: f_data/Healthy/12 (2).jpg  (18194 bytes)
 copying: f_data/Healthy/12.jpg  (18194 bytes)
 copying: f_data/Healthy/13 (2).jpg  (16537 bytes)
 copying: f_data/Healthy/13.jpg  (16537 bytes)
 copying: f_data/Healthy/14 (2).jpg  (16888 bytes)
 copying: f_data/Healthy/14.jpg  (16888 bytes)
 copying: f_data/Healthy/15 (2).jpg  (22342 bytes)
 copying: f_data/Healthy/15.jpg  (22342 bytes)
 copying: f_data/Healthy/16 (2).j

In [ ]:
# import zipfile

# with zipfile.ZipFile('/content/repaired_f_data.zip', 'r') as zip_ref:
#     zip_ref.extractall('/content/extracted_data')

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, models
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Flatten, Dense
# import os
# import cv2
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [ ]:
# def load_images_and_labels_binary(data_folder, img_size):
#     """
#     Loads images from 'healthy' and 'Redrot' subfolders.
#     Labels are 0 for 'healthy' and 1 for 'Redrot'.
#     """
#     images = []
#     labels = []
#     class_names = ['Healthy', 'RedRot']

#     for label_index, class_name in enumerate(class_names): # index 0 for healthy, 1 for Redrot
#         class_path = os.path.join(data_folder, class_name)
#         if os.path.isdir(class_path):
#             for image_name in os.listdir(class_path):
#                 image_path = os.path.join(class_path, image_name)
#                 try:
#                     img = cv2.imread(image_path)
#                     img = cv2.resize(img, (img_size, img_size))
#                     img = img / 255.0
#                     images.append(img)
#                     labels.append(label_index) # 0 for healthy, 1 for Redrot
#                 except Exception as e:
#                     print(f"Error loading image: {image_path} - {e}")

#     return np.array(images), np.array(labels), class_names

In [ ]:
# !pip install split-folders

In [ ]:
import splitfolders

input_folder = '/content/extracted_data/f_data'
output_folder = '/content/split_data'

splitfolders.ratio(input_folder, output=output_folder, ratio=(0.8, 0.2), seed=42)

Copying files: 2401 files [00:31, 76.31 files/s] 


In [ ]:
train_data_dir = '/content/split_data/train'
test_data_dir = '/content/split_data/val'
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
images, labels, class_names = load_images_and_labels_binary(train_data_dir,img_size= IMG_SIZE)

print(f"Loaded {len(images)} training images belonging to classes: {class_names}")

X_train, y_train, _ = load_images_and_labels_binary(train_data_dir, img_size=IMG_SIZE)
X_test, y_test, _ = load_images_and_labels_binary(test_data_dir, img_size=IMG_SIZE)

print(f"Training data shape: X_train: {X_train.shape}, y_train: {y_train.shape}")


def create_binary_cnn_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model


input_shape = (IMG_SIZE, IMG_SIZE, 3)
cnn_model = create_binary_cnn_model(input_shape)
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Loaded 1920 training images belonging to classes: ['Healthy', 'RedRot']
Training data shape: X_train: (1920, 128, 128, 3), y_train: (1920,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
cnn_model.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

cnn_model.summary()

print("\n--- Starting Model Training ---")
history = cnn_model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.2
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │       3,211,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,267,713 (12.47 MB)

 Trainable params: 3,267,713 (12.47 MB)

 Non-trainable params: 0 (0.00 B)


--- Starting Model Training ---
Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.6028 - loss: 0.8499 - val_accuracy: 0.0000e+00 - val_loss: 0.8067
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.6307 - loss: 0.6372 - val_accuracy: 0.1120 - val_loss: 1.0401
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7569 - loss: 0.4918 - val_accuracy: 0.7891 - val_loss: 0.4652
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8259 - loss: 0.3938 - val_accuracy: 0.7266 - val_loss: 0.5161
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8891 - loss: 0.3076 - val_accuracy: 0.6510 - val_loss: 0.6334
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8862 - loss: 0.2721 - val_accuracy: 0.7448 - val_loss: 0.4974
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9101 - loss: 0.2289 - val_accuracy: 0.7578 - val_loss: 0.4952
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9129 - lo

In [ ]:
y_pred_prob_test = cnn_model.predict(X_test)
y_pred_classes_test = (y_pred_prob_test > 0.5).astype(int)
y_true_classes_test = y_test

test_accuracy = accuracy_score(y_true_classes_test, y_pred_classes_test)
test_precision = precision_score(y_true_classes_test, y_pred_classes_test, average='binary')
test_recall = recall_score(y_true_classes_test, y_pred_classes_test, average='binary')
test_f1_score = f1_score(y_true_classes_test, y_pred_classes_test, average='binary')

print("\n--- Test Set Metrics ---")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1-score: {test_f1_score:.4f}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step

--- Test Set Metrics ---
Test Accuracy: 0.8711
Test Precision: 1.0000
Test Recall: 0.7427
Test F1-score: 0.8524


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("\n--- Starting Model Training ---")
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=BATCH_SIZE,
    validation_split=0.2
)


--- Starting Model Training ---
Epoch 1/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - accuracy: 0.6534 - loss: 1.0431 - val_accuracy: 0.7422 - val_loss: 0.5265
Epoch 2/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.8847 - loss: 0.2818 - val_accuracy: 0.9245 - val_loss: 0.2332
Epoch 3/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 91ms/step - accuracy: 0.9397 - loss: 0.1700 - val_accuracy: 0.5677 - val_loss: 0.9489
Epoch 4/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - accuracy: 0.9536 - loss: 0.1217 - val_accuracy: 0.8776 - val_loss: 0.3142
Epoch 5/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 0.9884 - loss: 0.0635 - val_accuracy: 0.8542 - val_loss: 0.3760


In [ ]:
y_pred_prob_test = model.predict(X_test)
y_pred_classes_test = (y_pred_prob_test > 0.5).astype(int)
y_true_classes_test = y_test

test_accuracy = accuracy_score(y_true_classes_test, y_pred_classes_test)
test_precision = precision_score(y_true_classes_test, y_pred_classes_test, average='binary')
test_recall = recall_score(y_true_classes_test, y_pred_classes_test, average='binary')
test_f1_score = f1_score(y_true_classes_test, y_pred_classes_test, average='binary')

print("\n--- Test Set Metrics ---")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1-score: {test_f1_score:.4f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 186ms/step

--- Test Set Metrics ---
Test Accuracy: 0.9044
Test Precision: 0.9665
Test Recall: 0.8382
Test F1-score: 0.8978


In [ ]:
from tensorflow.keras.applications import vgg19

In [ ]:
base_model = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("\n--- Starting Model Training ---")
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=BATCH_SIZE,
    validation_split=0.2
)


--- Starting Model Training ---
Epoch 1/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 9s 144ms/step - accuracy: 0.7119 - loss: 0.7528 - val_accuracy: 0.8672 - val_loss: 0.2960
Epoch 2/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 102ms/step - accuracy: 0.9477 - loss: 0.1678 - val_accuracy: 0.8906 - val_loss: 0.2382
Epoch 3/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 108ms/step - accuracy: 0.9736 - loss: 0.1104 - val_accuracy: 0.8411 - val_loss: 0.3896
Epoch 4/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 103ms/step - accuracy: 0.9853 - loss: 0.0719 - val_accuracy: 0.8438 - val_loss: 0.3425
Epoch 5/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.9798 - loss: 0.0741 - val_accuracy: 0.8125 - val_loss: 0.4956


In [ ]:
test_accuracy = accuracy_score(y_true_classes_test, y_pred_classes_test)
test_precision = precision_score(y_true_classes_test, y_pred_classes_test, average='binary')
test_recall = recall_score(y_true_classes_test, y_pred_classes_test, average='binary')
test_f1_score = f1_score(y_true_classes_test, y_pred_classes_test, average='binary')

print("\n--- Test Set Metrics ---")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1-score: {test_f1_score:.4f}")


--- Test Set Metrics ---
Test Accuracy: 0.9044
Test Precision: 0.9665
Test Recall: 0.8382
Test F1-score: 0.8978
